In [1]:
import os, subprocess
from pathlib import Path, PurePosixPath
import gzip
import json
import pandas as pd, numpy as np, pyranges as pr
import plotly.express as px
import clickhouse_connect

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.float_format', '{:.10f}'.format)

# UTR3

## Docs / notes

**Docs:**  
* [PyRanges v1.x GitHub](https://github.com/pyranges/pyranges_1.x) / [PyRanges v1.x Docs](https://pyranges1.readthedocs.io/en/latest/index.html)

**Notes:**  
 - !!! Нужно иметь ввиду, что позиции ФИЧ в GFF, например 3'-UTR, могут быть разделены на несколько интервалов, так как разделены, например, интронами.

**Sources**
 - [miRBase](https://www.mirbase.org): hsa.gff - human
 - [multiMiR](http://multimir.org)
 - [TarBase](https://dianalab.e-ce.uth.gr/tarbasev9/downloads)
 - [miRTarBase](https://awi.cuhk.edu.cn/~miRTarBase/miRTarBase_2025/php/index.php)
 - [MiRanda](http://mirtoolsgallery.tech/mirtoolsgallery/node/1055): ineractions prediction

**Tasks:**  
1. Привести все к одной нумерации позиций: GFF - 1-based, BED - 0-based
2. 
3. Проверить правильность offtarget_id. Сейчас это по факту позиции SV.

**Commands:**  
```{bash}
mamba env export -n utr3.venv > environment.yml
```

## Settings

In [3]:
main_path = Path.cwd()

In [4]:
# Create tree
refs_dir = main_path / 'data/refs'
gnomad_dir = main_path / 'data/gnomad'
rcmg_btk_sv_dir = main_path / 'data/rcmg_btk_sv'
clinvar_dir = main_path / 'data/clinvar'
mirna_dir = main_path / 'data/mirna'
mirbase_dir = mirna_dir / 'mirbase'
tarbase_dir = mirna_dir / 'tarbase'
other_dir = main_path / 'data/other'
output_dir = main_path / 'data/output'

Path(refs_dir).mkdir(parents=True, exist_ok=True)
Path(gnomad_dir).mkdir(parents=True, exist_ok=True)
Path(rcmg_btk_sv_dir).mkdir(parents=True, exist_ok=True)
Path(clinvar_dir).mkdir(parents=True, exist_ok=True)
Path(mirbase_dir).mkdir(parents=True, exist_ok=True)
Path(tarbase_dir).mkdir(parents=True, exist_ok=True)
Path(other_dir).mkdir(parents=True, exist_ok=True)
Path(output_dir).mkdir(parents=True, exist_ok=True)

## Functions

In [7]:
def fetch_file(link, output_dir):
    command = f'wget --no-clobber -P {output_dir} {link}'
    subprocess.run(command, shell=True)
    filename = PurePosixPath(link).name
    return output_dir / filename

def index_gff(input_filepath):
    command = f'igvtools index {input_filepath}'
    subprocess.run(command, shell=True)

def gunzip_file(input_filepath):
    output_filepath = input_filepath.with_suffix('')
    command = f'gunzip -c {input_filepath} > {output_filepath}'
    subprocess.run(command, shell=True)
    return output_filepath

def sort_gff(input_filepath):
    output_filepath = input_filepath.with_stem(input_filepath.stem + ".sorted")
    command = f'igvtools sort {input_filepath} {output_filepath}'
    subprocess.run(command, shell=True)
    return output_filepath

def index_gff(input_filepath):
    command = f'igvtools index {input_filepath}'
    subprocess.run(command, shell=True)

def define_join_type(row):
    feature_start = row['Start']
    feature_end = row['End']
    sv_start = row['Start_sv']
    sv_end = row['End_sv']

    join_type = np.nan
    if feature_start <= sv_start and feature_end >= sv_end: # SV полностью в feature
        join_type = 'sv_in_feature'
    elif sv_start <= feature_start and sv_end >= feature_end: # feature полностью в sv
        join_type = 'feature_in_sv'
    elif sv_start < feature_start and sv_end <= sv_end:
        join_type = 'sv_left_free'
    elif sv_start > feature_start and sv_end >= sv_end:
        join_type = 'sv_right_free'
    elif sv_start == feature_start and sv_end == sv_end:
        join_type = 'full_join'

    return join_type

## <span style="color:#00ff00;">Fetch data</span>

In [6]:
gff_filepath = fetch_file('https://ftp.ensembl.org/pub/release-115/gff3/homo_sapiens/Homo_sapiens.GRCh38.115.gff3.gz', refs_dir)
fasta_filepath = fetch_file('https://ftp.ensembl.org/pub/release-115/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.toplevel.fa.gz', refs_dir)
gnomad_sv_filepath = fetch_file('https://storage.googleapis.com/gcp-public-data--gnomad/release/4.1/genome_sv/gnomad.v4.1.sv.sites.bed.gz', gnomad_dir)
clinvar_filepath = fetch_file('https://ftp.ncbi.nlm.nih.gov/pub/clinvar/tab_delimited/variant_summary.txt.gz', clinvar_dir)
mirbase_filepath = fetch_file('https://www.mirbase.org/download/hsa.gff3', mirbase_dir)
tarbase_filepath = fetch_file('https://dianalab.e-ce.uth.gr/tarbasev9/data/Homo_sapiens_TarBase-v9.tsv.gz', tarbase_dir)

so_terms_filepath = fetch_file('https://raw.githubusercontent.com/The-Sequence-Ontology/SO-Ontologies/refs/heads/master/Ontology_Files/so.json', other_dir)

File '/Users/andrejnekrasov/pro/my/utr3/data/refs/Homo_sapiens.GRCh38.115.gff3.gz' already there; not retrieving.

File '/Users/andrejnekrasov/pro/my/utr3/data/refs/Homo_sapiens.GRCh38.dna.toplevel.fa.gz' already there; not retrieving.

File '/Users/andrejnekrasov/pro/my/utr3/data/gnomad/gnomad.v4.1.sv.sites.bed.gz' already there; not retrieving.

File '/Users/andrejnekrasov/pro/my/utr3/data/clinvar/variant_summary.txt.gz' already there; not retrieving.

File '/Users/andrejnekrasov/pro/my/utr3/data/mirna/mirbase/hsa.gff3' already there; not retrieving.

File '/Users/andrejnekrasov/pro/my/utr3/data/mirna/tarbase/Homo_sapiens_TarBase-v9.tsv.gz' already there; not retrieving.

File '/Users/andrejnekrasov/pro/my/utr3/data/other/so.json' already there; not retrieving.



In [7]:
# RCMG BTK SV
chromosomes = list(range(1, 23)) + ['X', 'Y', 'M']
rcmg_btk_sv_filepaths = []

try:
    client = clickhouse_connect.get_client(host='192.168.86.236', user='rdr', password='Dru_Todd3')
    db_name = "ngs"
    client.database = db_name
except:
    for chrom in chromosomes:
        output_filename =  f'rcmg_btk_sv_chr{chrom}.csv.gz'
        output_filepath = rcmg_btk_sv_dir / output_filename
        rcmg_btk_sv_filepaths.append(output_filepath)

        if not output_filepath.exists():
            rcmg_bkt_sv_df = client.query_df(f"SELECT * from ngs.GENOME_hg38_2_sv_chr{chrom} WHERE SVTYPE='DEL';")

            rcmg_bkt_sv_df['Chromosome'] = chrom
            rcmg_bkt_sv_df.to_csv(output_filepath, index=False, compression='gzip')
    try:
        client.close()
    except:
        pass

## <span style="color:#00ff00;">Prepare data</span>

In [7]:
gff_gunzipped_filepath = gunzip_file(gff_filepath)
gff_sorted_filepath = sort_gff(gff_gunzipped_filepath)
index_gff(gff_sorted_filepath)

Sorting /Users/andrejnekrasov/pro/my/utr3/data/refs/Homo_sapiens.GRCh38.115.gff3  -> /Users/andrejnekrasov/pro/my/utr3/data/refs/Homo_sapiens.GRCh38.115.sorted.gff3


Done
Done


### <span style="color:#00ff00;">Create PR/DF<span>

#### <span style="color:green;">Prepare GFF3 data</span>

In [ ]:
# GFF
gff_pr = pr.read_gff3(str(gff_filepath))

# ==============================================================================================================================================

# Add additional metrics
gff_pr['feature_len'] = (gff_pr['End'] - gff_pr['Start']).astype('int')
gff_pr['feature_id'] = 'feature_' + gff_pr['Chromosome'].astype('str') + ':' + gff_pr['Start'].astype('str') + '-' + gff_pr['End'].astype('str')

# Self annotation of GFF with mRNA (parent) data
target_columns = ['ID', 'Name', 'biotype', 'Parent', 'tag', 'transcript_support_level', 'feature_len', 'feature_id'] # Replace the data in these empty columns with mRNA data
annotation_gff_pr = gff_pr[target_columns].dropna(subset='ID')

annotation_suffix = '_annotation'
annotated_gff_pr = gff_pr.merge(annotation_gff_pr, how='left', left_on='Parent', right_on='ID', suffixes=['', annotation_suffix])

annotated_columns = [f'{column}{annotation_suffix}' for column in target_columns]
fill_dict = dict(zip(target_columns, annotated_columns))
for target, annotation in fill_dict.items():
    annotated_gff_pr[target] = annotated_gff_pr[target].fillna(annotated_gff_pr[annotation])
annotated_gff_pr = annotated_gff_pr.drop(annotated_columns, axis=1)

# ==============================================================================================================================================

# Create specific PRs (mRNAm, 3'-UTR, offtargets)
mrna_pr = annotated_gff_pr.query('Feature == "mRNA"')
three_utrs_pr = annotated_gff_pr.query('Feature == "three_prime_UTR"')

offtarget_features = ['five_prime_UTR',
                               'CDS',
                               #'lnc_RNA',
                               'ncRNA_gene',
                               #'snRNA',
                               #'snoRNA',
                               #'scRNA',
                               'rRNA',
                               'tRNA',
                               'processed_transcript',
                               ]
offtarget_features_pr = gff_pr.query('Feature in @offtarget_features')

# ==============================================================================================================================================

# 3'-UTR biotype stats
# Считаем именно уникальные вхождения, так как один 3'-UTR может быть разбить на несколько интервалов в GFF (разделен интронами).
# В особенности это выражено у nonsense_mediated_decay транскриптов (нам они не пригодятся, но тем не менее).
three_utrs_pr.drop_duplicates(subset='ID')['biotype'].value_counts().to_frame().reset_index()

# ==============================================================================================================================================

# Keep only MANE_select / MANE_Select|Ensembl_canonical
three_utrs_pr_filtered = three_utrs_pr[three_utrs_pr['tag'].str.contains('MANE_Select', na=False, regex=True)]

# Stats
# MANE_select genes count = 19437 (release_1.5) from summary file: https://ftp.ncbi.nlm.nih.gov/refseq/MANE/MANE_human/release_1.5/
three_utrs_pr_filtered['biotype'].value_counts().to_frame().reset_index()

#### <span style="color:green;">gnomAD SV</span>

In [ ]:
# gnomAD SV
# Keep only the necessary columns (63 of 600...) in SV
with gzip.open(str(gnomad_sv_filepath), 'rt') as sv_file:
    sv_header = sv_file.readline().strip().split('\t')

target_columns = ['#chrom', 'start', 'end', 'name', 'svtype', 'samples', 'MULTIALLELIC', 'ALGORITHMS', 'BOTHSIDES_SUPPORT', 'CHR2', 'CPX_INTERVALS', 'CPX_TYPE', 'END', 'END2', 'EVIDENCE', 'LOW_CONFIDENCE_REPETITIVE_LARGE_DUP', 'MEMBERS', 'NCR', 'OUTLIER_SAMPLE_ENRICHED_LENIENT', 'PAR', 'PCRMINUS_NCR', 'PCRPLUS_NCR', 'PESR_GT_OVERDISPERSION', 'POS2', 'PREDICTED_BREAKEND_EXONIC', 'PREDICTED_COPY_GAIN', 'PREDICTED_DUP_PARTIAL', 'PREDICTED_INTERGENIC', 'PREDICTED_INTRAGENIC_EXON_DUP', 'PREDICTED_INTRONIC', 'PREDICTED_INV_SPAN', 'PREDICTED_LOF', 'PREDICTED_MSV_EXON_OVERLAP', 'PREDICTED_NEAREST_TSS', 'PREDICTED_NONCODING_BREAKPOINT', 'PREDICTED_NONCODING_SPAN', 'PREDICTED_PARTIAL_DISPERSED_DUP', 'PREDICTED_PARTIAL_EXON_DUP', 'PREDICTED_PROMOTER', 'PREDICTED_TSS_DUP', 'PREDICTED_UTR', 'RESOLVED_POSTHOC', 'SOURCE', 'SVLEN', 'SVTYPE', 'UNRESOLVED_TYPE', 'AN', 'AC', 'AF', 'N_BI_GENOS', 'N_HOMREF', 'N_HET', 'N_HOMALT', 'FREQ_HOMREF', 'FREQ_HET', 'FREQ_HOMALT', 'CN_NUMBER', 'CN_COUNT', 'CN_STATUS', 'CN_FREQ', 'CN_NONREF_COUNT', 'CN_NONREF_FREQ', 'FILTER']
target_columns_indexes = [sv_header.index(i) for i in target_columns]
target_columns[0:3] = ['Chromosome', 'Start', 'End'] # Rename columns

sv_df = pd.read_csv(str(gnomad_sv_filepath), sep='\t', usecols=target_columns_indexes, names=target_columns, comment='#')
sv_df['Chromosome'] = sv_df['Chromosome'].str.replace('chr', '')

sv_pr = pr.PyRanges(sv_df)

# ==============================================================================================================================================

# Keep target SVs (DEL)
sv_targets = ['DEL']
sv_filter = ['PASS']
target_sv_pr = sv_pr.query('svtype in @sv_targets and FILTER in @sv_filter')

# Add columns
target_sv_pr['sv_len'] = target_sv_pr.End - target_sv_pr.Start
target_sv_pr['sv_id'] = 'sv_' + target_sv_pr['Chromosome'].astype('str') + ':' + target_sv_pr['Start'].astype('str') + '-' + target_sv_pr['End'].astype('str')

# ==============================================================================================================================================

# SV percentiles by AF
target_sv_pr['AF'].describe(percentiles=[.1, .25, .5, .75, .9, .95, .99, .999]).to_frame().reset_index()

# SV percentiles by SV length
target_sv_pr['sv_len'].describe(percentiles=[.1, .25, .5, .75, .9, .95, .99, .999]).to_frame().reset_index()

/var/folders/rf/k3pdmpqx0bjfvwyn2q6sw5v00000gn/T/ipykernel_75092/1131200204.py:10: DtypeWarning: Columns (10,11,24,25,26,28,29,30,31,32,33,36,37,38,39,40,42,57,58,59) have mixed types. Specify dtype option on import or set low_memory=False.
  sv_df = pd.read_csv(str(gnomad_sv_filepath), sep='\t', usecols=target_columns_indexes, names=target_columns, comment='#')


#### <span style="color:green;">ClinVar</span>

In [ ]:
# ClinVar
clinvar_df = pd.read_csv(clinvar_filepath, sep='\t', compression='gzip')

# ==============================================================================================================================================

clinvar_df = clinvar_df.replace(['na', '-'], np.nan)
clinvar_df = clinvar_df.rename(columns={'Stop': 'End'})
clinvar_df_filtered = clinvar_df.query('Assembly == "GRCh38"')

/var/folders/rf/k3pdmpqx0bjfvwyn2q6sw5v00000gn/T/ipykernel_75092/1108798041.py:2: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  clinvar_df = pd.read_csv(clinvar_filepath, sep='\t', compression='gzip')


#### <span style="color:green;">RCMG BTK SV</span>

In [ ]:
# RCMG BTK SV
rcmg_btk_sv_filepath = rcmg_btk_sv_dir / 'rcmg_btk_sv.csv.gz'

if not rcmg_btk_sv_filepath.exists():
    rcmg_bkt_sv_df = pd.DataFrame() # concated filtered aggregated
    for rcmg_btk_sv_file in rcmg_btk_sv_filepaths:
        print(rcmg_btk_sv_file)
        current_rcmg_bkt_sv_df = pd.read_csv(str(rcmg_btk_sv_file), compression='gzip')

        # Add columns
        current_rcmg_bkt_sv_df['AF'] = current_rcmg_bkt_sv_df['AD'] / current_rcmg_bkt_sv_df['DP']
        current_rcmg_bkt_sv_df['SVLEN'] = abs(current_rcmg_bkt_sv_df['END'] - current_rcmg_bkt_sv_df['POS'])

        # Drop columns
        current_rcmg_bkt_sv_df = current_rcmg_bkt_sv_df.drop(columns=['REF', 'ALT'])

        # Filter
        current_rcmg_bkt_sv_df_filtered = current_rcmg_bkt_sv_df.query('caller == "manta"')

        # Aggregate
        current_rcmg_bkt_sv_df_filtered_aggregated = current_rcmg_bkt_sv_df_filtered.groupby(['Chromosome', 'POS', 'END', 'SVTYPE', 'caller', 'SVLEN'], as_index=False).agg({
            'SID': [list, 'count'],
            'DP': [list, 'min', 'median', 'mean', 'max'],
            'AD': [list, 'min', 'median', 'mean', 'max'],
            'AF': [list, 'min', 'median', 'mean', 'max']
        })

        # Rename columns
        current_rcmg_bkt_sv_df_filtered_aggregated.columns = ['_'.join(col).strip('_') for col in current_rcmg_bkt_sv_df_filtered_aggregated.columns]
        current_rcmg_bkt_sv_df_filtered_aggregated = current_rcmg_bkt_sv_df_filtered_aggregated.rename(columns={'POS': 'Start', 'END': 'End'})
        current_rcmg_bkt_sv_df_filtered_aggregated = current_rcmg_bkt_sv_df_filtered_aggregated.reset_index(drop=True)

        # Concat
        rcmg_bkt_sv_df = pd.concat([rcmg_bkt_sv_df, current_rcmg_bkt_sv_df_filtered_aggregated], ignore_index=False)
    rcmg_bkt_sv_df.to_csv(str(rcmg_btk_sv_filepath), index=False, compression='gzip')
else:
    rcmg_bkt_sv_df = pd.read_csv(str(rcmg_btk_sv_filepath), compression='gzip')

#### <span style="color:green;">miRBase</span>

In [14]:
# miRBase
mirbase_pr = pr.read_gff3(str(mirbase_filepath))
mirbase_pr['Chromosome'] = mirbase_pr['Chromosome'].str.replace('chr', '')

#### <span style="color:green;">TarBase</span>

In [15]:
# TarBase
tarbase_df = pd.read_csv(str(tarbase_filepath), sep='\t', compression='gzip')
tarbase_df = tarbase_df.rename(columns={'chromosome': 'Chromosome', 'start': 'Start', 'end': 'End', 'strand': 'Strand'})
tarbase_df['Chromosome'] = tarbase_df['Chromosome'].str.replace('chr', '')

tarbase_pr = pr.PyRanges(tarbase_df)
tarbase_pr = tarbase_pr.dropna(subset=['Chromosome', 'Start', 'End'], how='any')
tarbase_pr[['Start', 'End']] = tarbase_pr[['Start', 'End']].astype('int64')

/var/folders/rf/k3pdmpqx0bjfvwyn2q6sw5v00000gn/T/ipykernel_75092/1318660838.py:2: DtypeWarning: Columns (5,14,15,19,21) have mixed types. Specify dtype option on import or set low_memory=False.
  tarbase_df = pd.read_csv(str(tarbase_filepath), sep='\t', compression='gzip')
/var/folders/rf/k3pdmpqx0bjfvwyn2q6sw5v00000gn/T/ipykernel_75092/1318660838.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tarbase_pr[['Start', 'End']] = tarbase_pr[['Start', 'End']].astype('int64')


#### <span style="color:green;">Sequence Ontology terms</span>

In [16]:
# SO terms
with open(so_terms_filepath, 'r') as so_terms_file:
    so_terms = json.load(so_terms_file)

so_terms_filtered = {
    'lbl': [],
    'definition': []
}
for i in so_terms['graphs'][0]['nodes']:
    lbl = i.get('lbl', np.nan)
    definition = i.get('meta', {}).get('definition', {}).get('val', np.nan)
    so_terms_filtered['lbl'].append(lbl)
    so_terms_filtered['definition'].append(definition)

so_terms_df = pd.DataFrame.from_dict(so_terms_filtered)
so_terms_df['lbl'] = so_terms_df['lbl'].str.capitalize().replace('_', '')

### <span style="color:#00ff00;">EDA (Exploratory Data Analysis)</span>

In [17]:
# GFF features stats
gff_features = gff_pr.Feature.value_counts().to_frame().reset_index()
gff_features['tmp_id'] = gff_features['Feature'].str.capitalize().replace('_', '')
gff_features = gff_features.merge(so_terms_df, how='left', left_on='tmp_id', right_on='lbl')
gff_features = gff_features.drop(['lbl', 'tmp_id'], axis=1)
pd.DataFrame(gff_features)

,Feature,count,definition
0,exon,3683354,A region of the transcript sequence within a gene which is not removed from the primary RNA transcript by RNA splicing.
1,CDS,2284414,"A contiguous sequence which begins with, and includes, a start codon and ends with, and includes, a stop codon."
2,five_prime_UTR,426629,A region at the 5' end of a mature transcript (preceding the initiation codon) that is not translated into a protein.
3,three_prime_UTR,340592,A region at the 3' end of a mature transcript (following the stop codon) that is not translated into a protein.
4,mRNA,233603,Messenger RNA is the intermediate molecule between DNA and protein. It includes UTR and coding sequences. It does not contain introns.
5,lnc_RNA,225629,NaN
6,biological_region,182818,A region defined by its disposition to be involved in a biological process.
7,ncRNA_gene,42124,A gene that encodes a non-coding RNA.
8,transcript,28808,An RNA synthesized on a DNA or RNA template by an RNA polymerase.
9,gene,21571,"A region (or regions) that includes all of the sequence elements necessary to encode a functional transcript. A gene may include regulatory regions, transcribed regions and/or other functional sequence regions."


In [18]:
# GFF biotypes stats
gff_biotypes = gff_pr.biotype.value_counts().to_frame().reset_index()
gff_biotypes['tmp_id'] = gff_biotypes['biotype'].str.capitalize().replace('_', '')
gff_biotypes = gff_biotypes.merge(so_terms_df, how='left', left_on='tmp_id', right_on='lbl')
gff_biotypes = gff_biotypes.drop(['lbl', 'tmp_id'], axis=1)
pd.DataFrame(gff_biotypes)

,biotype,count,definition
0,protein_coding,231596,"A gene which, when transcribed, can be translated into a protein."
1,lncRNA,226412,"A non-coding RNA generally longer than 200 nucleotides that cannot be classified as any other ncRNA subtype. Similar to mRNAs, lncRNAs are mainly transcribed by RNA polymerase II, are often capped by 7-methyl guanosine at their 5' ends, polyadenylated at their 3' ends and may be spliced."
2,retained_intron,34239,NaN
3,protein_coding_CDS_not_defined,26573,NaN
4,nonsense_mediated_decay,21949,NaN
5,processed_pseudogene,18979,"A pseudogene created via retrotranposition of the mRNA of a functional protein-coding parent gene followed by accumulation of deleterious mutations lacking introns and promoters, often including a polyA tail."
6,misc_RNA,4432,NaN
7,unprocessed_pseudogene,3898,NaN
8,snRNA,3820,A small nuclear RNA molecule involved in pre-mRNA splicing and processing.
9,miRNA,3758,"Small, ~22-nt, RNA molecule that is the endogenous transcript of a miRNA gene (or the product of other non coding RNA genes). Micro RNAs are produced from precursor molecules (SO:0001244) that can form local hairpin structures, which ordinarily are processed (usually via the Dicer pathway) such that a single miRNA molecule accumulates from one arm of a hairpin precursor molecule. Micro RNAs may trigger the cleavage of their target molecules or act as translational repressors."


In [19]:
sv_pr.svtype.value_counts().to_frame().reset_index()

,svtype,count
0,DEL,1197080
1,BND,356035
2,DUP,269326
3,INS:ME:ALU,173374
4,INS,83441
5,INS:ME:LINE1,30223
6,INS:ME:SVA,17607
7,CPX,15189
8,DEL:ME:LINE1,8505
9,INV,2193


In [20]:
tarbase_pr['gene_location'].value_counts()

gene_location
CDS     2502793
3UTR    2056551
5UTR        175
Name: count, dtype: int64

### <span style="color:#00ff00;">Prepare GFF3 data</span>

In [21]:
# Add additional metrics
gff_pr['feature_len'] = (gff_pr['End'] - gff_pr['Start']).astype('int')
gff_pr['feature_id'] = 'feature_' + gff_pr['Chromosome'].astype('str') + ':' + gff_pr['Start'].astype('str') + '-' + gff_pr['End'].astype('str')

# Self annotation of GFF with mRNA (parent) data
target_columns = ['ID', 'Name', 'biotype', 'Parent', 'tag', 'transcript_support_level', 'feature_len', 'feature_id'] # Replace the data in these empty columns with mRNA data
annotation_gff_pr = gff_pr[target_columns].dropna(subset='ID')

annotation_suffix = '_annotation'
annotated_gff_pr = gff_pr.merge(annotation_gff_pr, how='left', left_on='Parent', right_on='ID', suffixes=['', annotation_suffix])

annotated_columns = [f'{column}{annotation_suffix}' for column in target_columns]
fill_dict = dict(zip(target_columns, annotated_columns))
for target, annotation in fill_dict.items():
    annotated_gff_pr[target] = annotated_gff_pr[target].fillna(annotated_gff_pr[annotation])
annotated_gff_pr = annotated_gff_pr.drop(annotated_columns, axis=1)

In [22]:
# Create specific PRs (mRNAm, 3'-UTR, offtargets)
mrna_pr = annotated_gff_pr.query('Feature == "mRNA"')
three_utrs_pr = annotated_gff_pr.query('Feature == "three_prime_UTR"')

offtarget_features = ['five_prime_UTR',
                               'CDS',
                               #'lnc_RNA',
                               'ncRNA_gene',
                               #'snRNA',
                               #'snoRNA',
                               #'scRNA',
                               'rRNA',
                               'tRNA',
                               'processed_transcript',
                               ]
offtarget_features_pr = gff_pr.query('Feature in @offtarget_features')

In [23]:
# 3'-UTR biotype stats
# Считаем именно уникальные вхождения, так как один 3'-UTR может быть разбить на несколько интервалов в GFF (разделен интронами).
# В особенности это выражено у nonsense_mediated_decay транскриптов (нам они не пригодятся, но тем не менее).
three_utrs_pr.drop_duplicates(subset='ID')['biotype'].value_counts().to_frame().reset_index()


,biotype,count
0,protein_coding,190278
1,nonsense_mediated_decay,21947
2,protein_coding_LoF,43
3,IG_C_gene,22
4,TR_C_gene,6
5,IG_V_gene,2


In [24]:
# Keep only MANE_select / MANE_Select|Ensembl_canonical
three_utrs_pr_filtered = three_utrs_pr[three_utrs_pr['tag'].str.contains('MANE_Select', na=False, regex=True)]

# Stats
# MANE_select genes count = 19437 (release_1.5) from summary file: https://ftp.ncbi.nlm.nih.gov/refseq/MANE/MANE_human/release_1.5/
three_utrs_pr_filtered['biotype'].value_counts().to_frame().reset_index()

,biotype,count
0,protein_coding,19757


### <span style="color:#00ff00;">Prepare SV data</span>

#### <span style="color:green;">gnomAD</span>

In [25]:
# Keep target SVs (DEL)
sv_targets = ['DEL']
sv_filter = ['PASS']
target_sv_pr = sv_pr.query('svtype in @sv_targets and FILTER in @sv_filter')

# Add columns
target_sv_pr['sv_len'] = target_sv_pr.End - target_sv_pr.Start
target_sv_pr['sv_id'] = 'sv_' + target_sv_pr['Chromosome'].astype('str') + ':' + target_sv_pr['Start'].astype('str') + '-' + target_sv_pr['End'].astype('str')

/var/folders/rf/k3pdmpqx0bjfvwyn2q6sw5v00000gn/T/ipykernel_75092/2433410726.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_sv_pr['sv_len'] = target_sv_pr.End - target_sv_pr.Start
/var/folders/rf/k3pdmpqx0bjfvwyn2q6sw5v00000gn/T/ipykernel_75092/2433410726.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_sv_pr['sv_id'] = 'sv_' + target_sv_pr['Chromosome'].astype('str') + ':' + target_sv_pr['Start'].astype('str') + '-' + target_sv_pr['End'].astype('str')


In [26]:
# SV percentiles by AF
target_sv_pr['AF'].describe(percentiles=[.1, .25, .5, .75, .9, .95, .99, .999]).to_frame().reset_index()

,index,AF
0,count,627321.0000000000
1,mean,0.0045937636
2,std,0.0415649697
3,min,0.0000080000
4,10%,0.0000080000
5,25%,0.0000080000
6,50%,0.0000240000
7,75%,0.0001110000
8,90%,0.0009820000
9,95%,0.0033650000


In [27]:
# SV percentiles by SV length
target_sv_pr['sv_len'].describe(percentiles=[.1, .25, .5, .75, .9, .95, .99, .999]).to_frame().reset_index()

,index,sv_len
0,count,627321.0000000000
1,mean,8529.8884988706
2,std,139720.7127188754
3,min,51.0000000000
4,10%,76.0000000000
5,25%,137.0000000000
6,50%,702.0000000000
7,75%,4073.0000000000
8,90%,10309.0000000000
9,95%,23359.0000000000


In [28]:
'''fig = px.histogram( 
    x=target_sv_pr['sv_len'],
    nbins=300,
    log_y=True,
    title='gnomAD SV DEL lenght distribution (log)'
)
fig.show()'''

"fig = px.histogram( \n    x=target_sv_pr['sv_len'],\n    nbins=300,\n    log_y=True,\n    title='gnomAD SV DEL lenght distribution (log)'\n)\nfig.show()"

#### <span style="color:green;">RCMG BTK SV</span>

In [10]:
rcmg_btk_sv_filepath = rcmg_btk_sv_dir / 'rcmg_btk_sv.csv.gz'

if not rcmg_btk_sv_filepath.exists():
    rcmg_bkt_sv_df = pd.DataFrame() # concated filtered aggregated
    for rcmg_btk_sv_file in rcmg_btk_sv_filepaths:
        print(rcmg_btk_sv_file)
        current_rcmg_bkt_sv_df = pd.read_csv(str(rcmg_btk_sv_file), compression='gzip')

        # Add columns
        current_rcmg_bkt_sv_df['AF'] = current_rcmg_bkt_sv_df['AD'] / current_rcmg_bkt_sv_df['DP']
        current_rcmg_bkt_sv_df['SVLEN'] = abs(current_rcmg_bkt_sv_df['END'] - current_rcmg_bkt_sv_df['POS'])

        # Drop columns
        current_rcmg_bkt_sv_df = current_rcmg_bkt_sv_df.drop(columns=['REF', 'ALT'])

        # Filter
        current_rcmg_bkt_sv_df_filtered = current_rcmg_bkt_sv_df.query('caller == "manta"')

        # Aggregate
        current_rcmg_bkt_sv_df_filtered_aggregated = current_rcmg_bkt_sv_df_filtered.groupby(['Chromosome', 'POS', 'END', 'SVTYPE', 'caller', 'SVLEN'], as_index=False).agg({
            'SID': [list, 'count'],
            'DP': [list, 'min', 'median', 'mean', 'max'],
            'AD': [list, 'min', 'median', 'mean', 'max'],
            'AF': [list, 'min', 'median', 'mean', 'max']
        })

        # Rename columns
        current_rcmg_bkt_sv_df_filtered_aggregated.columns = ['_'.join(col).strip('_') for col in current_rcmg_bkt_sv_df_filtered_aggregated.columns]
        current_rcmg_bkt_sv_df_filtered_aggregated = current_rcmg_bkt_sv_df_filtered_aggregated.rename(columns={'POS': 'Start', 'END': 'End'})
        current_rcmg_bkt_sv_df_filtered_aggregated = current_rcmg_bkt_sv_df_filtered_aggregated.reset_index(drop=True)

        # Concat
        rcmg_bkt_sv_df = pd.concat([rcmg_bkt_sv_df, current_rcmg_bkt_sv_df_filtered_aggregated], ignore_index=False)
    rcmg_bkt_sv_df.to_csv(str(rcmg_btk_sv_filepath), index=False, compression='gzip')
else:
    rcmg_bkt_sv_df = pd.read_csv(str(rcmg_btk_sv_filepath), compression='gzip')

/var/folders/rf/k3pdmpqx0bjfvwyn2q6sw5v00000gn/T/ipykernel_90049/1031382406.py:36: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  rcmg_bkt_sv_df = pd.read_csv(str(rcmg_btk_sv_filepath), compression='gzip')


In [14]:
rcmg_bkt_sv_df.query('SID_count < 5').sort_values(by='SID_count', ascending=False)

,Chromosome,Start,End,SVTYPE,caller,SVLEN,SID_list,SID_count,DP_list,DP_min,DP_median,DP_mean,DP_max,AD_list,AD_min,AD_median,AD_mean,AD_max,AF_list,AF_min,AF_median,AF_mean,AF_max
3028568,16,89587758,89587887,DEL,manta,129,"['000007121080', '000007090090', '000007146070', '000007092940']",4,"[15, 10, 7, 21]",7,12.5000000000,13.2500000000,21,"[10, 7, 7, 15]",7,8.5000000000,9.7500000000,15,"[0.6666666666666666, 0.7, 1.0, 0.7142857142857143]",0.6666666667,0.7071428571,0.7702380952,1.0000000000
3757127,X,151763379,151763512,DEL,manta,133,"['000027027560', '000027027610', '000007425420', '000027023500']",4,"[4, 3, 3, 8]",3,3.5000000000,4.5000000000,8,"[4, 2, 2, 4]",2,3.0000000000,3.0000000000,4,"[1.0, 0.6666666666666666, 0.6666666666666666, 0.5]",0.5000000000,0.6666666667,0.7083333333,1.0000000000
2688245,13,17086759,17146613,DEL,manta,59854,"['000007070820', '000007405730', '000029004370', '000029001870']",4,"[10, 10, 19, 13]",10,11.5000000000,13.0000000000,19,"[5, 7, 10, 10]",5,8.5000000000,8.0000000000,10,"[0.5, 0.7, 0.5263157894736842, 0.7692307692307693]",0.5000000000,0.6131578947,0.6238866397,0.7692307692
2688247,13,17086759,17146615,DEL,manta,59856,"['000007073710', '000007177110', '000007184240', '000007416750']",4,"[23, 17, 14, 16]",14,16.5000000000,17.5000000000,23,"[13, 11, 8, 12]",8,11.5000000000,11.0000000000,13,"[0.5652173913043478, 0.6470588235294118, 0.5714285714285714, 0.75]",0.5652173913,0.6092436975,0.6334261966,0.7500000000
1070691,5,46659146,46708885,DEL,manta,49739,"['000007075930', '000007147100', '000007168650', '000007186100']",4,"[30, 33, 24, 32]",24,31.0000000000,29.7500000000,33,"[28, 31, 17, 28]",17,28.0000000000,26.0000000000,31,"[0.9333333333333333, 0.9393939393939394, 0.7083333333333334, 0.875]",0.7083333333,0.9041666667,0.8640151515,0.9393939394
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1441101,6,160507372,160659872,DEL,manta,152500,['000007151860'],1,[63],63,63.0000000000,63.0000000000,63,[20],20,20.0000000000,20.0000000000,20,[0.31746031746031744],0.3174603175,0.3174603175,0.3174603175,0.3174603175
1441104,6,160507372,160659882,DEL,manta,152510,['000007139390'],1,[53],53,53.0000000000,53.0000000000,53,[17],17,17.0000000000,17.0000000000,17,[0.32075471698113206],0.3207547170,0.3207547170,0.3207547170,0.3207547170
1441106,6,160507372,160659895,DEL,manta,152523,['000007408380'],1,[54],54,54.0000000000,54.0000000000,54,[11],11,11.0000000000,11.0000000000,11,[0.2037037037037037],0.2037037037,0.2037037037,0.2037037037,0.2037037037
1441107,6,160507373,160659825,DEL,manta,152452,['000007181600'],1,[57],57,57.0000000000,57.0000000000,57,[27],27,27.0000000000,27.0000000000,27,[0.47368421052631576],0.4736842105,0.4736842105,0.4736842105,0.4736842105


### <span style="color:#00ff00;">Join GFF3 x gnomAD SV</span>

In [29]:
# Join GFF x SV
features_sv_joined_pr = three_utrs_pr_filtered.join_overlaps(target_sv_pr, suffix='_sv', report_overlap_column='join_len')

In [30]:
# Add additional columns (join_start, join_end, join_id)
features_sv_joined_pr['join_start'] = np.maximum(features_sv_joined_pr['Start'], features_sv_joined_pr['Start_sv'])
features_sv_joined_pr['join_end'] = np.minimum(features_sv_joined_pr['End'], features_sv_joined_pr['End_sv'])
features_sv_joined_pr['join_id'] = 'join_' + features_sv_joined_pr['Chromosome'].astype('str') + ':' + features_sv_joined_pr['join_start'].astype('str') + '-' + features_sv_joined_pr['join_end'].astype('str')

In [31]:
# Define joint type
features_sv_joined_pr['join_type'] = features_sv_joined_pr.apply(define_join_type, axis=1)
features_sv_joined_pr['join_type'].value_counts().to_frame().reset_index()

,join_type,count
0,feature_in_sv,17788
1,sv_in_feature,3610
2,sv_right_free,2129
3,sv_left_free,1975


In [32]:
# Count joints by SV
joints_count_by_sv = features_sv_joined_pr['sv_id'].value_counts().to_frame().reset_index()
joints_count_by_sv = joints_count_by_sv.rename(columns={'count': 'joins_count'})

# Add joints count data
features_sv_joined_pr = features_sv_joined_pr.merge(joints_count_by_sv, how='left')
#len(features_sv_joined_pr)

# Filter joins by join type
targey_joint_types = ['sv_in_feature', 'sv_right_free', 'sv_left_free']
features_sv_joined_pr_filtered = features_sv_joined_pr.query('join_type in @targey_joint_types and ((Strand == "+" and join_type == "sv_right_free") or (Strand == "-" and join_type == "sv_left_free") or join_type == "sv_in_feature") and joins_count == 1')

#### <span style="color:green;">Join type stats</span>

In [33]:
# Stats by join type
features_sv_joined_pr_filtered.groupby('join_type')['sv_len'].describe(percentiles=[.1, .25, .5, .75, .9, .95, .99, .999]).T

join_type,sv_in_feature,sv_left_free,sv_right_free
count,3521.0000000000,962.0000000000,1124.0000000000
mean,508.7157057654,7150.2079002079,7269.1227758007
std,928.4987400891,18421.3678622167,19884.5691740781
min,51.0000000000,52.0000000000,53.0000000000
10%,61.0000000000,656.6000000000,628.3000000000
25%,83.0000000000,1627.2500000000,1584.0000000000
50%,167.0000000000,4131.5000000000,3872.0000000000
75%,570.0000000000,6911.5000000000,7001.0000000000
90%,1226.0000000000,11954.3000000000,12049.1000000000
95%,1965.0000000000,22855.2500000000,20279.4000000000


In [34]:
features_sv_joined_pr_filtered['biotype'].value_counts().to_frame().reset_index()

,biotype,count
0,protein_coding,5607


### <span style="color:#00ff00;">Offtarget features exclusion</span>

#### <span style="color:green;">features_sv_joined_pr_filtered copy</span>

In [35]:
# Offtarget stats
features_sv_joined_pr_filtered_cp = features_sv_joined_pr_filtered.copy()
features_sv_joined_pr_filtered_cp[['Start_tmp', 'End_tmp']] = features_sv_joined_pr_filtered_cp[['Start', 'End']] # Save old positions
features_sv_joined_pr_filtered_cp[['Start', 'End']] = features_sv_joined_pr_filtered_cp[['Start_sv', 'End_sv']] # SV positions to main positions

#### <span style="color:green;">Filter offtargets</span>

In [36]:
# Exclude offtargets (SV x offtarget overlaps)
features_sv_no_offtereget_joined_pr = features_sv_joined_pr_filtered_cp.overlap(offtarget_features_pr, invert=True, strand_behavior='ignore')

features_sv_no_offtereget_joined_pr[['Start', 'End']] = features_sv_no_offtereget_joined_pr[['Start_tmp', 'End_tmp']]
features_sv_no_offtereget_joined_pr = features_sv_no_offtereget_joined_pr.drop(['Start_tmp', 'End_tmp'], axis=1)

len(features_sv_no_offtereget_joined_pr)

4154

In [37]:
# Joint type stats after offtargets exclution
print('Before offtargets filtering')
print(features_sv_joined_pr_filtered['join_type'].value_counts().to_frame().reset_index())
print('\nAfter offtargets filtering')
print(features_sv_no_offtereget_joined_pr['join_type'].value_counts().to_frame().reset_index())

Before offtargets filtering
       join_type  count
0  sv_in_feature   3521
1  sv_right_free   1124
2   sv_left_free    962

After offtargets filtering
       join_type  count
0  sv_in_feature   2834
1  sv_right_free    702
2   sv_left_free    618


#### <span style="color:green;">Offtarget stats</span>

In [38]:
# features_sv_offtereget_joined_pr - только для отслеживания пересечений с offtarget. Фильтрованный от offtarget pr - features_sv_no_offtereget_joined_pr (через overlap, потому что join находя пересечения с offtarget (через left) также оставляет оригинальную строку).DS_Storefeatures_sv_offtereget_joined_pr = features_sv_joined_pr_filtered_cp.join_overlaps(offtarget_features_pr, join_type='left', suffix='_offtarget', report_overlap_column='offtarget_join_len')
features_sv_offtereget_joined_pr = features_sv_joined_pr_filtered_cp.join_overlaps(offtarget_features_pr, join_type='left', suffix='_offtarget', report_overlap_column='offtarget_join_len', strand_behavior='ignore')

features_sv_offtereget_joined_pr[['Start', 'End']] = features_sv_offtereget_joined_pr[['Start_tmp', 'End_tmp']]
features_sv_offtereget_joined_pr = features_sv_offtereget_joined_pr.drop(['Start_tmp', 'End_tmp'], axis=1)

features_sv_offtereget_joined_pr['offtarget_id'] = 'offtarget_' + features_sv_offtereget_joined_pr['Chromosome'].astype('str') + ':' + features_sv_offtereget_joined_pr['Start_offtarget'].astype('str') + '-' + features_sv_offtereget_joined_pr['End_offtarget'].astype('str')

In [39]:
# Features offtargets stats
# Суммарные значения данной статистики будут выше чем разница всех джоинов - оффтартет джоинов, так как один и тот же SV (напр. DEL) могут пересекать разные изоформы одного и того же транскрипта.
features_sv_offtereget_joined_pr.groupby(['join_type', 'Feature_offtarget'])['Feature_offtarget'].agg(['count'])
#features_sv_offtereget_joined_pr.drop_duplicates(subset='sv_id').groupby(['join_type', 'Feature_offtarget'])['Feature_offtarget'].agg(['count'])

count
join_type     Feature_offtarget          
sv_in_feature CDS                     164
              five_prime_UTR           19
              ncRNA_gene              656
              processed_transcript      4
sv_left_free  CDS                    1458
              five_prime_UTR         1015
              ncRNA_gene              320
sv_right_free CDS                    1903
              five_prime_UTR          813
              ncRNA_gene              416

### <span style="color:#00ff00;">miRNA join</span>

#### <span style="color:green;">features_sv_no_offtereget_joined_pr copy</span>

In [40]:
# Offtarget stats
features_sv_no_offtereget_joined_pr_cp = features_sv_no_offtereget_joined_pr.copy()
features_sv_no_offtereget_joined_pr_cp[['Start_tmp', 'End_tmp']] = features_sv_no_offtereget_joined_pr_cp[['Start', 'End']] # Save old positions
features_sv_no_offtereget_joined_pr_cp[['Start', 'End']] = features_sv_no_offtereget_joined_pr_cp[['join_start', 'join_end']] # SV positions to main positions

#### <span style="color:green;">Join</span>

In [41]:
# Join TarBase
features_sv_no_offtereget_tarbase_joined_pr = features_sv_no_offtereget_joined_pr_cp.join_overlaps(tarbase_pr, suffix='_tarbase', strand_behavior='ignore', report_overlap_column='tarbase_join_len')

features_sv_no_offtereget_tarbase_joined_pr[['Start', 'End']] = features_sv_no_offtereget_tarbase_joined_pr[['Start_tmp', 'End_tmp']]
features_sv_no_offtereget_tarbase_joined_pr = features_sv_no_offtereget_tarbase_joined_pr.drop(['Start_tmp', 'End_tmp'], axis=1)

features_sv_no_offtereget_tarbase_joined_pr['tarbase_id'] = 'tarbase_' + features_sv_no_offtereget_tarbase_joined_pr['Chromosome'].astype('str') + ':' + features_sv_no_offtereget_tarbase_joined_pr['Start_tarbase'].astype('str') + '-' + features_sv_no_offtereget_tarbase_joined_pr['End_tarbase'].astype('str') + ':' + features_sv_no_offtereget_tarbase_joined_pr['mirna_name']

In [42]:
features_sv_no_offtereget_tarbase_joined_pr['AF'].describe(percentiles=[.1, .25, .5, .75, .9, .95, .99, .999]).to_frame().reset_index()

,index,AF
0,count,90375.0000000000
1,mean,0.0002522849
2,std,0.0032934593
3,min,0.0000080000
4,10%,0.0000080000
5,25%,0.0000080000
6,50%,0.0000160000
7,75%,0.0000560000
8,90%,0.0002540000
9,95%,0.0008310000


In [43]:
features_sv_no_offtereget_tarbase_joined_pr['gene_location'].value_counts()

gene_location
3UTR    90373
CDS         2
Name: count, dtype: int64

In [44]:
features_sv_no_offtereget_tarbase_joined_pr['interaction_group'].value_counts()

interaction_group
primary      67005
secondary    23370
Name: count, dtype: int64

In [45]:
features_sv_no_offtereget_tarbase_joined_pr['regulation'].value_counts()

regulation
Negative    90375
Name: count, dtype: int64

In [46]:
features_sv_no_offtereget_tarbase_joined_pr['transcript_name'].value_counts().to_frame().reset_index().to_csv(output_dir / 'mirna_genes.csv', index=False)
features_sv_no_offtereget_tarbase_joined_pr['transcript_name'].value_counts().to_frame().reset_index()

,transcript_name,count
0,YOD1-202,982
1,SDE2-201,976
2,ZBTB10-203,953
3,PTEN-220,911
4,KATNAL1-201,773
...,...,...
1536,ZBTB10-201,1
1537,MS4A1-201,1
1538,MMP16-201,1
1539,CYTH2-203,1


In [47]:
features_sv_no_offtereget_tarbase_joined_pr['experimental_method'].value_counts()

experimental_method
HITS-CLIP                    56089
PAR-CLIP                     33412
Chimeric fragments             390
qCLASH                         257
Luciferase Reporter Assay      191
CLASH                           28
IMPACT-Seq                       5
3LIFE                            3
Name: count, dtype: int64

In [48]:
features_sv_no_offtereget_tarbase_joined_pr['join_type'].value_counts()

join_type
sv_in_feature    40369
sv_right_free    29495
sv_left_free     20511
Name: count, dtype: int64

In [49]:
features_sv_no_offtereget_tarbase_joined_pr['join_id'].value_counts()

join_id
join_10:87966367-87971930     1098
join_1:207043848-207048293     971
join_8:80519631-80525632       785
join_3:86067499-86074429       750
join_13:30202629-30207106      725
                              ... 
join_14:57469299-57470464        1
join_6:56056873-56056976         1
join_6:43615332-43616937         1
join_6:41599219-41599319         1
join_X:155777999-155782459       1
Name: count, Length: 2290, dtype: int64

In [50]:
pd.DataFrame(features_sv_no_offtereget_tarbase_joined_pr)

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,ID,Alias,external_name,logic_name,Name,biotype,description,gene_id,version,Parent,transcript_id,constitutive,ensembl_end_phase,ensembl_phase,exon_id,rank,tag,transcript_support_level,ccdsid,protein_id,feature_len,feature_id,Start_sv,End_sv,name,svtype,samples,MULTIALLELIC,ALGORITHMS,BOTHSIDES_SUPPORT,CHR2,CPX_INTERVALS,CPX_TYPE,END,END2,EVIDENCE,LOW_CONFIDENCE_REPETITIVE_LARGE_DUP,MEMBERS,NCR,OUTLIER_SAMPLE_ENRICHED_LENIENT,PAR,PCRMINUS_NCR,PCRPLUS_NCR,PESR_GT_OVERDISPERSION,POS2,PREDICTED_BREAKEND_EXONIC,PREDICTED_COPY_GAIN,PREDICTED_DUP_PARTIAL,PREDICTED_INTERGENIC,PREDICTED_INTRAGENIC_EXON_DUP,PREDICTED_INTRONIC,PREDICTED_INV_SPAN,PREDICTED_LOF,PREDICTED_MSV_EXON_OVERLAP,PREDICTED_NEAREST_TSS,PREDICTED_NONCODING_BREAKPOINT,PREDICTED_NONCODING_SPAN,PREDICTED_PARTIAL_DISPERSED_DUP,PREDICTED_PARTIAL_EXON_DUP,PREDICTED_PROMOTER,PREDICTED_TSS_DUP,PREDICTED_UTR,RESOLVED_POSTHOC,SOURCE,SVLEN,SVTYPE,UNRESOLVED_TYPE,AN,AC,AF,N_BI_GENOS,N_HOMREF,N_HET,N_HOMALT,FREQ_HOMREF,FREQ_HET,FREQ_HOMALT,CN_NUMBER,CN_COUNT,CN_STATUS,CN_FREQ,CN_NONREF_COUNT,CN_NONREF_FREQ,FILTER,sv_len,sv_id,join_len,join_start,join_end,join_id,join_type,joins_count,species,mirna_name,mirna_id,gene_name,gene_id_tarbase,gene_location,transcript_name,transcript_id_tarbase,Start_tarbase,End_tarbase,Strand_tarbase,experimental_method,regulation,tissue,cell_line,article_pubmed_id,confidence,interaction_group,cell_type,microt_score,comment,tarbase_join_len,tarbase_id
4,1,ensembl_havana,three_prime_UTR,1427787,1430255,.,+,.,transcript:ENST00000378821,NaN,NaN,NaN,TMEM88B-201,protein_coding,NaN,NaN,NaN,transcript:ENST00000378821,NaN,NaN,NaN,NaN,NaN,NaN,"gencode_basic,gencode_primary,Ensembl_canonical,MANE_Select",2 (assigned to previous version 3),NaN,NaN,2468,feature_1:1427787-1430255,1429037,1429179,gnomAD-SV_v3_DEL_chr1_08f4439e,DEL,NaN,False,manta,True,chr1,NaN,NaN,1429179,NaN,SR,False,NaN,0.0019674799,False,False,0.0019242300,0.0024584001,False,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TMEM88B,False,NaN,141,DEL,NaN,125770.0000000000,1.0000000000,0.0000080000,62885.0000000000,62884.0000000000,1.0000000000,0.0000000000,0.9999840260,0.0000159020,0.0000000000,NaN,NaN,NaN,NaN,NaN,NaN,PASS,142,sv_1:1429037-1429179,142,1429037,1429179,join_1:1429037-1429179,sv_in_feature,1,Homo sapiens,hsa-let-7a-5p,MIMAT0000062,TMEM88B,ENSG00000205116,3UTR,TMEM88B-201,ENST00000378821,1429042,1429054,+,HITS-CLIP,Negative,Brain - Cingulate gyrus,NaN,24389009.0000000000,1,primary,NaN,NaN,Male - 44 years,12,tarbase_1:1429042-1429054:hsa-let-7a-5p
4,1,ensembl_havana,three_prime_UTR,1427787,1430255,.,+,.,transcript:ENST00000378821,NaN,NaN,NaN,TMEM88B-201,protein_coding,NaN,NaN,NaN,transcript:ENST00000378821,NaN,NaN,NaN,NaN,NaN,NaN,"gencode_basic,gencode_primary,Ensembl_canonical,MANE_Select",2 (assigned to previous version 3),NaN,NaN,2468,feature_1:1427787-1430255,1429037,1429179,gnomAD-SV_v3_DEL_chr1_08f4439e,DEL,NaN,False,manta,True,chr1,NaN,NaN,1429179,NaN,SR,False,NaN,0.0019674799,False,False,0.0019242300,0.0024584001,False,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TMEM88B,False,NaN,141,DEL,NaN,125770.0000000000,1.0000000000,0.0000080000,62885.0000000000,62884.0000000000,1.0000000000,0.0000000000,0.9999840260,0.0000159020,0.0000000000,NaN,NaN,NaN,NaN,NaN,NaN,PASS,142,sv_1:1429037-1429179,142,1429037,1429179,join_1:1429037-1429179,sv_in_feature,1,Homo sapiens,hsa-let-7b-5p,MIMAT0000063,TMEM88B,ENSG00000205116,3UTR,TMEM88B-201,ENST00000378821,1429042,1429054,+,HITS-CLIP,Negative,Brain - Cingulate gyrus,NaN,24389009.0000000000,1,secondary,NaN,NaN,Male - 44 years,12,tarbase_1:1429042-1429054:hsa-let-7b-5p
4,1,ensembl_havana,three_prime_UTR,1427787,1430255,.,+,.,transcript:ENST00000378821,NaN,NaN,NaN,TMEM88B-201,protein_coding,NaN,NaN,NaN,transcript:ENST00000378821,NaN,NaN,NaN,NaN,NaN,NaN,"gencode_basic,gencode_primary,Ensembl_canonical,MANE_Select",2 (assigned to previous version 3),Na

### <span style="color:#00ff00;">GFF output for IGV</span>

In [51]:
pr_for_igv = features_sv_no_offtereget_tarbase_joined_pr

In [52]:
pd.DataFrame(pr_for_igv)

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,ID,Alias,external_name,logic_name,Name,biotype,description,gene_id,version,Parent,transcript_id,constitutive,ensembl_end_phase,ensembl_phase,exon_id,rank,tag,transcript_support_level,ccdsid,protein_id,feature_len,feature_id,Start_sv,End_sv,name,svtype,samples,MULTIALLELIC,ALGORITHMS,BOTHSIDES_SUPPORT,CHR2,CPX_INTERVALS,CPX_TYPE,END,END2,EVIDENCE,LOW_CONFIDENCE_REPETITIVE_LARGE_DUP,MEMBERS,NCR,OUTLIER_SAMPLE_ENRICHED_LENIENT,PAR,PCRMINUS_NCR,PCRPLUS_NCR,PESR_GT_OVERDISPERSION,POS2,PREDICTED_BREAKEND_EXONIC,PREDICTED_COPY_GAIN,PREDICTED_DUP_PARTIAL,PREDICTED_INTERGENIC,PREDICTED_INTRAGENIC_EXON_DUP,PREDICTED_INTRONIC,PREDICTED_INV_SPAN,PREDICTED_LOF,PREDICTED_MSV_EXON_OVERLAP,PREDICTED_NEAREST_TSS,PREDICTED_NONCODING_BREAKPOINT,PREDICTED_NONCODING_SPAN,PREDICTED_PARTIAL_DISPERSED_DUP,PREDICTED_PARTIAL_EXON_DUP,PREDICTED_PROMOTER,PREDICTED_TSS_DUP,PREDICTED_UTR,RESOLVED_POSTHOC,SOURCE,SVLEN,SVTYPE,UNRESOLVED_TYPE,AN,AC,AF,N_BI_GENOS,N_HOMREF,N_HET,N_HOMALT,FREQ_HOMREF,FREQ_HET,FREQ_HOMALT,CN_NUMBER,CN_COUNT,CN_STATUS,CN_FREQ,CN_NONREF_COUNT,CN_NONREF_FREQ,FILTER,sv_len,sv_id,join_len,join_start,join_end,join_id,join_type,joins_count,species,mirna_name,mirna_id,gene_name,gene_id_tarbase,gene_location,transcript_name,transcript_id_tarbase,Start_tarbase,End_tarbase,Strand_tarbase,experimental_method,regulation,tissue,cell_line,article_pubmed_id,confidence,interaction_group,cell_type,microt_score,comment,tarbase_join_len,tarbase_id
4,1,ensembl_havana,three_prime_UTR,1427787,1430255,.,+,.,transcript:ENST00000378821,NaN,NaN,NaN,TMEM88B-201,protein_coding,NaN,NaN,NaN,transcript:ENST00000378821,NaN,NaN,NaN,NaN,NaN,NaN,"gencode_basic,gencode_primary,Ensembl_canonical,MANE_Select",2 (assigned to previous version 3),NaN,NaN,2468,feature_1:1427787-1430255,1429037,1429179,gnomAD-SV_v3_DEL_chr1_08f4439e,DEL,NaN,False,manta,True,chr1,NaN,NaN,1429179,NaN,SR,False,NaN,0.0019674799,False,False,0.0019242300,0.0024584001,False,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TMEM88B,False,NaN,141,DEL,NaN,125770.0000000000,1.0000000000,0.0000080000,62885.0000000000,62884.0000000000,1.0000000000,0.0000000000,0.9999840260,0.0000159020,0.0000000000,NaN,NaN,NaN,NaN,NaN,NaN,PASS,142,sv_1:1429037-1429179,142,1429037,1429179,join_1:1429037-1429179,sv_in_feature,1,Homo sapiens,hsa-let-7a-5p,MIMAT0000062,TMEM88B,ENSG00000205116,3UTR,TMEM88B-201,ENST00000378821,1429042,1429054,+,HITS-CLIP,Negative,Brain - Cingulate gyrus,NaN,24389009.0000000000,1,primary,NaN,NaN,Male - 44 years,12,tarbase_1:1429042-1429054:hsa-let-7a-5p
4,1,ensembl_havana,three_prime_UTR,1427787,1430255,.,+,.,transcript:ENST00000378821,NaN,NaN,NaN,TMEM88B-201,protein_coding,NaN,NaN,NaN,transcript:ENST00000378821,NaN,NaN,NaN,NaN,NaN,NaN,"gencode_basic,gencode_primary,Ensembl_canonical,MANE_Select",2 (assigned to previous version 3),NaN,NaN,2468,feature_1:1427787-1430255,1429037,1429179,gnomAD-SV_v3_DEL_chr1_08f4439e,DEL,NaN,False,manta,True,chr1,NaN,NaN,1429179,NaN,SR,False,NaN,0.0019674799,False,False,0.0019242300,0.0024584001,False,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TMEM88B,False,NaN,141,DEL,NaN,125770.0000000000,1.0000000000,0.0000080000,62885.0000000000,62884.0000000000,1.0000000000,0.0000000000,0.9999840260,0.0000159020,0.0000000000,NaN,NaN,NaN,NaN,NaN,NaN,PASS,142,sv_1:1429037-1429179,142,1429037,1429179,join_1:1429037-1429179,sv_in_feature,1,Homo sapiens,hsa-let-7b-5p,MIMAT0000063,TMEM88B,ENSG00000205116,3UTR,TMEM88B-201,ENST00000378821,1429042,1429054,+,HITS-CLIP,Negative,Brain - Cingulate gyrus,NaN,24389009.0000000000,1,secondary,NaN,NaN,Male - 44 years,12,tarbase_1:1429042-1429054:hsa-let-7b-5p
4,1,ensembl_havana,three_prime_UTR,1427787,1430255,.,+,.,transcript:ENST00000378821,NaN,NaN,NaN,TMEM88B-201,protein_coding,NaN,NaN,NaN,transcript:ENST00000378821,NaN,NaN,NaN,NaN,NaN,NaN,"gencode_basic,gencode_primary,Ensembl_canonical,MANE_Select",2 (assigned to previous version 3),Na

In [53]:
# Add GFF
output_features_pr = pr_for_igv[['Chromosome', 'Feature', 'Start', 'End', 'Strand', 'Name', 'biotype', 'tag', 'ID']]
output_features_pr = output_features_pr.drop_duplicates()
output_features_pr['Source'] = 'GFF'
output_features_pr['color'] = 'blue'

# Add gnomAD SV
output_sv_pr = pr_for_igv[['Chromosome', 'Start_sv', 'End_sv', 'svtype', 'name', 'sv_id', 'AF', 'N_HET', 'N_HOMALT']]
output_sv_pr = output_sv_pr.drop_duplicates()
output_sv_pr['Source'] = 'gnomAD_SV'
output_sv_pr['color'] = 'red'
output_sv_pr = pr.PyRanges(output_sv_pr.rename(columns={'Start_sv': 'Start', 'End_sv': 'End', 'svtype': 'Feature', 'name': 'Name', 'sv_id': 'ID'}))

# Add joins data
output_joins_pr = pr_for_igv[['Chromosome', 'join_start', 'join_end', 'join_type', 'join_id']]
output_joins_pr = output_joins_pr.drop_duplicates()
output_joins_pr['Source'] = 'join'
output_joins_pr['color'] = 'orange'
output_joins_pr = pr.PyRanges(output_joins_pr.rename(columns={'join_start': 'Start', 'join_end': 'End', 'join_type': 'Feature', 'join_id': 'ID'}))

# Add miRNA TarBase
output_tarbase_pr = pr_for_igv[['Chromosome', 'Start_tarbase', 'End_tarbase', 'Strand_tarbase', 'mirna_name', 'interaction_group', 'mirna_id', 'tarbase_id']]
output_tarbase_pr = output_tarbase_pr.drop_duplicates(subset=['mirna_id', 'tarbase_id'])
output_tarbase_pr['Source'] = 'tarbase'
output_tarbase_pr['color'] = 'green'
output_tarbase_pr = pr.PyRanges(output_tarbase_pr.rename(columns={'Start_tarbase': 'Start', 'End_tarbase': 'End', 'Strand_tarbase': 'Strand', 'interaction_group': 'Feature', 'mirna_name': 'Name', 'tarbase_id': 'ID'}))

# Add offtarget GFF (features) data
output_offtargets_pr = features_sv_offtereget_joined_pr[['Chromosome', 'Start_offtarget', 'End_offtarget', 'Feature_offtarget', 'offtarget_id']]
output_offtargets_pr = output_offtargets_pr.query('Start_offtarget == Start_offtarget')
output_offtargets_pr = output_offtargets_pr.drop_duplicates()
output_offtargets_pr['Source'] = 'GFF_offtarget'
output_offtargets_pr['color'] = '#d9d9d9'
output_offtargets_pr = pr.PyRanges(output_offtargets_pr.rename(columns={'Start_offtarget': 'Start', 'End_offtarget': 'End', 'Feature_offtarget': 'Feature', 'offtarget_id': 'ID'}))

output_igv_pr = pd.concat([output_features_pr, output_sv_pr, output_joins_pr, output_tarbase_pr, output_offtargets_pr], ignore_index=True)

/var/folders/rf/k3pdmpqx0bjfvwyn2q6sw5v00000gn/T/ipykernel_75092/1204189949.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output_features_pr['Source'] = 'GFF'
/var/folders/rf/k3pdmpqx0bjfvwyn2q6sw5v00000gn/T/ipykernel_75092/1204189949.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output_features_pr['color'] = 'blue'


In [54]:
# Stats
output_igv_pr['Source'].value_counts().to_frame().reset_index()

,Source,count
0,tarbase,41904
1,GFF_offtarget,2921
2,gnomAD_SV,2294
3,join,2290
4,GFF,1669


In [55]:
# Fill NA and save
output_igv_pr = output_igv_pr.reset_index(drop=True)
#output_igv_pr = pr.PyRanges(output_igv_pr.fillna('.'))

output_igv_pr.to_gff3(output_dir / 'regions_for_igv.gff3')